# Image format 

Objective is to understand better what docker images are.

## Lab setup

This is done via Jupyter, see setup here: https://github.com/scoulomb/misc-notes/blob/master/Jupyter/README.md


## Build

We will build and save toto image.
In this image we add a file, add a second file, declare and enviromnment var and remove the first file.


In [95]:
cd ~/image_format
docker build -t toto -f toto.Dockerfile .

Sending build context to Docker daemon  3.072kB
Step 1/5 : FROM python
 ---> 930516bcf910
Step 2/5 : RUN touch /toto_rm.txt
 ---> Using cache
 ---> baeab0eb566e
Step 3/5 : RUN touch /toto.txt
 ---> Using cache
 ---> f012b37a5a35
Step 4/5 : ENV tutu toto
 ---> Using cache
 ---> 83750f1b5354
Step 5/5 : RUN rm -f toto_rm.txt
 ---> Using cache
 ---> 9836ad22cf3f
Successfully built 9836ad22cf3f
Successfully tagged toto:latest


: 1

In [96]:
rm -rf ~/tmp_image_format
mkdir  ~/tmp_image_format
docker save toto >  ~/tmp_image_format/toto.tar

: 1

In [97]:
ls -l  ~/tmp_image_format/toto.tar

-rw-rw-r-- 1 scoulomb scoulomb 941896704 août  16 14:42 /home/scoulomb/tmp_image_format/toto.tar


: 1

## Docker image structure

We can uncompress the tarball and observe the content.


In [98]:
# untar the file
rm -rf  ~/tmp_image_format/toto_tar_uncompress
mkdir ~/tmp_image_format/toto_tar_uncompress
tar -xf ~/tmp_image_format/toto.tar -C ~/tmp_image_format/toto_tar_uncompress
tree  ~/tmp_image_format/toto_tar_uncompress # ll  ~/tmp_image_format/toto_tar_uncompress

/home/scoulomb/tmp_image_format/toto_tar_uncompress
├── 22f0d7c7bfdfcb59806e14be3e83d85f92a3bb7604e072922b109eaedd84100b
│   ├── json
│   ├── layer.tar
│   └── VERSION
├── 300b272b0214adc3c331f206bb0b863fdbfac74276ebb51412eca4b2ae5ffae9
│   ├── json
│   ├── layer.tar
│   └── VERSION
├── 54ff2caf5f99d9b81c3a0b5acee2107941c3075490d2f8abf8b718a24d567e93
│   ├── json
│   ├── layer.tar
│   └── VERSION
├── 7f24e93677d0c5f66a08ecf936dd73d47fc14c75e28c18c2d03181bcdbe912a5
│   ├── json
│   ├── layer.tar
│   └── VERSION
├── 8844a0ee96bb354a4033cc08797d144eeee1ee3c8c2ae2682e78459b38279bd1
│   ├── json
│   ├── layer.tar
│   └── VERSION
├── 8ed0edc8e61650e2ef4969d954822efb39304884d46c898864398185d58ffa12
│   ├── json
│   ├── layer.tar
│   └── VERSION
├── 9836ad22cf3fcab17b4ad58ba324bc06dd518986346ee5313e42d3dfa0a769da.json
├── aa93f3a87b7fa38394c5550145bdc9debf3915b860ea509444df8eb5ccd58a72
│   ├── json
│   ├── layer.tar
│   └── VERSION
├── b6d06422210ef192cf5f56712d7284fd45121d86a79ea9cc3c89fc2928

: 1

On `manifest.json` we have the details of each layer. It is pointing to a `layer` folder contained in same archive.

In [99]:
cat ~/tmp_image_format/toto_tar_uncompress/manifest.json | jq

[
  {
    "Config": "9836ad22cf3fcab17b4ad58ba324bc06dd518986346ee5313e42d3dfa0a769da.json",
    "RepoTags": [
      "toto:latest"
    ],
    "Layers": [
      "300b272b0214adc3c331f206bb0b863fdbfac74276ebb51412eca4b2ae5ffae9/layer.tar",
      "aa93f3a87b7fa38394c5550145bdc9debf3915b860ea509444df8eb5ccd58a72/layer.tar",
      "e6be1d18b1e08a70a496a691e5aba4a0c5d5999db6a155af14ad222232e8be67/layer.tar",
      "c7908f9982db9bd94c7f6ee7f4a3ca72b9341c81c1029a2119d6082a96d8d2f1/layer.tar",
      "b6d06422210ef192cf5f56712d7284fd45121d86a79ea9cc3c89fc2928d7ce42/layer.tar",
      "8844a0ee96bb354a4033cc08797d144eeee1ee3c8c2ae2682e78459b38279bd1/layer.tar",
      "fd82dabe7bf1a1a3310eb40654709ea6a7efd1d9d7b844394558b306629677c5/layer.tar",
      "f2fdb508d5ca36888d959b93f2083c418ff80b746ea45b0f90586260e24e78e3/layer.tar",
      "8ed0edc8e61650e2ef4969d954822efb39304884d46c898864398185d58ffa12/layer.tar",
      "7f24e93677d0c5f66a08ecf936dd73d47fc14c75e28c18c2d03181bcdbe912a5/layer.tar",
      

: 1

In [100]:
cat ~/tmp_image_format/toto_tar_uncompress/repositories

{"toto":{"latest":"22f0d7c7bfdfcb59806e14be3e83d85f92a3bb7604e072922b109eaedd84100b"}}


: 1

While repositories contains info of last layer and in particular tagging information, and sha of last layer.
Equivalent to 

In [101]:
cat ~/tmp_image_format/toto_tar_uncompress/manifest.json | jq --raw-output '.[0].Layers[-1]'

22f0d7c7bfdfcb59806e14be3e83d85f92a3bb7604e072922b109eaedd84100b/layer.tar


: 1

## Exploring layers

### Methods (optional read)

#### Method 1: use repositories (works only for first layer)

In [102]:
rm -rf ~/tmp_image_format/layer_1_uncompress
mkdir ~/tmp_image_format/layer_1_uncompress
tar -xf  ~/tmp_image_format/toto_tar_uncompress/$(cat ~/tmp_image_format/toto_tar_uncompress/repositories | jq .toto.latest | tr -d '"')/layer.tar -C ~/tmp_image_format/layer_1_uncompress
ll ~/tmp_image_format/layer_1_uncompress

total 8h
drwxrwxr-x 2 scoulomb scoulomb 4096 août  16 14:42 ./
drwxrwxr-x 4 scoulomb scoulomb 4096 août  16 14:42 ../
-rw------- 1 scoulomb scoulomb    0 août  16 12:38 .wh.toto_rm.txt


: 1

#### Method 2: use manifest file

In [103]:
rm -rf ~/tmp_image_format/layer_1_uncompress
mkdir ~/tmp_image_format/layer_1_uncompress

# export card_layers=$(cat ~/tmp_image_format/toto_tar_uncompress/manifest.json | jq -r '.[0].Layers | length')
# echo "Number of layers"
# echo $(expr $card_layers) => Use https://github.com/stedolan/jq/issues/509

tar -xf  ~/tmp_image_format/toto_tar_uncompress/$(cat ~/tmp_image_format/toto_tar_uncompress/manifest.json | jq  '.[0].Layers[-1]' | tr -d '"') -C ~/tmp_image_format/layer_1_uncompress
ll ~/tmp_image_format/layer_1_uncompress

total 8h
drwxrwxr-x 2 scoulomb scoulomb 4096 août  16 14:43 ./
drwxrwxr-x 4 scoulomb scoulomb 4096 août  16 14:43 ../
-rw------- 1 scoulomb scoulomb    0 août  16 12:38 .wh.toto_rm.txt


: 1

#### Method 3: use manifest file and JQ the rigth way

In [104]:
LAST_LAYER=$(jq --raw-output '.[0].Layers[-1]'  ~/tmp_image_format/toto_tar_uncompress/manifest.json | sed 's/\/layer.tar//g')
echo $LAST_LAYER

rm -rf ~/tmp_image_format/layer_1_uncompress
mkdir ~/tmp_image_format/layer_1_uncompress

tar -xf  ~/tmp_image_format/toto_tar_uncompress/$LAST_LAYER/layer.tar -C ~/tmp_image_format/layer_1_uncompress
ll ~/tmp_image_format/layer_1_uncompress


22f0d7c7bfdfcb59806e14be3e83d85f92a3bb7604e072922b109eaedd84100b
total 8h
drwxrwxr-x 2 scoulomb scoulomb 4096 août  16 14:43 ./
drwxrwxr-x 4 scoulomb scoulomb 4096 août  16 14:43 ../
-rw------- 1 scoulomb scoulomb    0 août  16 12:38 .wh.toto_rm.txt


: 1

### Layers



In [105]:
for i in {1..13..1}
do
    echo -e "========================================================================================================"

    export LAYER_INDEX=-$i
    LAYER=$(jq --raw-output ".[0].Layers[$LAYER_INDEX]"  ~/tmp_image_format/toto_tar_uncompress/manifest.json | sed 's/\/layer.tar//g')

    echo "== Showing layer $LAYER with index $LAYER_INDEX"

    echo "=== json"
    cat  ~/tmp_image_format/toto_tar_uncompress/$LAYER/json  | jq 

    echo "=== VERSION"
    cat  ~/tmp_image_format/toto_tar_uncompress/$LAYER/VERSION 

    echo -e "\n"

    echo "=== Tarball content"

    rm -rf ~/tmp_image_format/layer_uncompress/$LAYER_INDEX
    mkdir -p ~/tmp_image_format/layer_uncompress/$LAYER_INDEX

    tar -xf  ~/tmp_image_format/toto_tar_uncompress/$LAYER/layer.tar -C ~/tmp_image_format/layer_uncompress/$LAYER_INDEX
    ll  ~/tmp_image_format/layer_uncompress/$LAYER_INDEX

done



== Showing layer 22f0d7c7bfdfcb59806e14be3e83d85f92a3bb7604e072922b109eaedd84100b with index -1
=== json
{
  "id": "22f0d7c7bfdfcb59806e14be3e83d85f92a3bb7604e072922b109eaedd84100b",
  "parent": "54ff2caf5f99d9b81c3a0b5acee2107941c3075490d2f8abf8b718a24d567e93",
  "created": "2022-08-16T10:38:11.774795417Z",
  "container": "786d45c32c339075ada5e9b7b4b431eaa386d13ceec9ec0812078865c64cec5c",
  "container_config": {
    "Hostname": "",
    "Domainname": "",
    "User": "",
    "AttachStdin": false,
    "AttachStdout": false,
    "AttachStderr": false,
    "Tty": false,
    "OpenStdin": false,
    "StdinOnce": false,
    "Env": [
      "PATH=/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin",
      "LANG=C.UTF-8",
      "GPG_KEY=A035C8C19219BA821ECEA86B64E628F8D684696D",
      "PYTHON_VERSION=3.10.5",
      "PYTHON_PIP_VERSION=22.0.4",
      "PYTHON_SETUPTOOLS_VERSION=58.1.0",
      "PYTHON_GET_PIP_URL=https://github.com/pypa/get-pip/raw/6ce3639da143c5d79b44f94b04

: 1

## Observations and take-away

- We have alayer per command (RUN, COPY, ADD) and not a layer per dockerfile
This is specific to docker, some other images building tool may have a diffrent behavior
For example: https://nixos.org/guides/building-and-running-docker-images.html

- Each layer reflect a state of file system.
Note the `.wh.toto_rm.txt`.
See docker without: https://github.com/moby/moby/blob/master/image/spec/v1.md#creating-an-image-filesystem-changeset






- Env var are added in last layer (created by Docker)

In [106]:
export LAYER_INDEX=-1
LAYER=$(jq --raw-output ".[0].Layers[$LAYER_INDEX]"  ~/tmp_image_format/toto_tar_uncompress/manifest.json | sed 's/\/layer.tar//g')
cat  ~/tmp_image_format/toto_tar_uncompress/$LAYER/json  | jq .container_config.Env

[
  "PATH=/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin",
  "LANG=C.UTF-8",
  "GPG_KEY=A035C8C19219BA821ECEA86B64E628F8D684696D",
  "PYTHON_VERSION=3.10.5",
  "PYTHON_PIP_VERSION=22.0.4",
  "PYTHON_SETUPTOOLS_VERSION=58.1.0",
  "PYTHON_GET_PIP_URL=https://github.com/pypa/get-pip/raw/6ce3639da143c5d79b44f94b04080abf2531fd6e/public/get-pip.py",
  "PYTHON_GET_PIP_SHA256=ba3ab8267d91fd41c58dbce08f76db99f747f716d85ce1865813842bb035524d",
  "tutu=toto"
]


: 1

- Docker image are arhcitecture and OS specific (other layer do not mention archi in metadata, but I assume there is one). Cf [what-is-docker](./what-is-docker.md)

In [107]:
cat  ~/tmp_image_format/toto_tar_uncompress/$LAYER/json  | jq .architecture
cat  ~/tmp_image_format/toto_tar_uncompress/$LAYER/json  | jq .os

"amd64"
"linux"


: 1

From observation we can start giving a definition to Docker [see next section](./what-is-docker.md).

<!-- sha seems consistent across rebuild,
 export via rigth clik file explo downalod and markdown export -->